In [1]:
# 20200127

# Talked with Yoram about setting control for small segments (any seg <3.6 K no need to set control, since the control will be lower than 100 ~2.5%)
3600*0.028

100.8

In [2]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import paramiko
import random
print(datetime.datetime.now())
os.getcwd()

2020-01-28 15:14:05.755805


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1'

In [54]:
df_0_to_18_detail=pd.read_csv("./final_details_output_2020-01-27/BL_2020Q1_0_to_18_details_JL_2020-01-27.csv",
                        dtype=str,usecols=['customer_id_hashed','email_address_hash','segment','zip_type','HML_Group'])
df_test=df_0_to_18_detail[((df_0_to_18_detail['zip_type']=="T") & (df_0_to_18_detail['HML_Group']=="H"))]
df_test.shape


(897488, 5)

In [3]:
df_0_to_18_detail=pd.read_csv("./final_details_output_2020-01-27/BL_2020Q1_0_to_18_details_JL_2020-01-27.csv",
                        dtype=str,usecols=['customer_id_hashed','email_address_hash','segment','zip_type'])
df_19_to_48_detail=pd.read_csv("./final_details_output_2020-01-27/BL_2020Q1_19_to_48_details_JL_2020-01-27.csv",
                        dtype=str,usecols=['customer_id_hashed','email_address_hash','segment','zip_type'])
df_0_to_18_detail=df_0_to_18_detail[df_0_to_18_detail['zip_type'].isin(["P","S"])]
df_19_to_48_detail=df_19_to_48_detail[df_19_to_48_detail['zip_type']=="P"]

del df_0_to_18_detail['zip_type']
del df_19_to_48_detail['zip_type']
print("df_0_to_18_detail.shape",df_0_to_18_detail.shape)
print("df_19_to_48_detail.shape",df_19_to_48_detail.shape)


df_0_to_18_detail.shape (17854401, 3)
df_19_to_48_detail.shape (4915668, 3)


In [29]:
df_0_to_18_count_all=df_0_to_18_detail.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_19_to_48_count_all=df_19_to_48_detail.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_0_to_18_count_all=df_0_to_18_count_all.sort_values("segment")
df_19_to_48_count_all=df_19_to_48_count_all.sort_values("segment")

list_seg_small_0_to_18=df_0_to_18_count_all[df_0_to_18_count_all['customer_id_hashed']<=3600]['segment'].unique().tolist()
list_seg_small_19_to_48=df_19_to_48_count_all[df_19_to_48_count_all['customer_id_hashed']<=3600]['segment'].unique().tolist()

df_0_to_18_detail_test_only=df_0_to_18_detail[df_0_to_18_detail['segment'].isin(list_seg_small_0_to_18)]
df_0_to_18_detail_large=df_0_to_18_detail[~df_0_to_18_detail['segment'].isin(list_seg_small_0_to_18)]

df_19_to_48_detail_test_only=df_19_to_48_detail[df_19_to_48_detail['segment'].isin(list_seg_small_19_to_48)]
df_19_to_48_detail_large=df_19_to_48_detail[~df_19_to_48_detail['segment'].isin(list_seg_small_19_to_48)]

print("df_0_to_18_detail_test_only.shape",df_0_to_18_detail_test_only.shape)
print("df_19_to_48_detail_test_only.shape",df_19_to_48_detail_test_only.shape)

print("df_0_to_18_detail_large.shape",df_0_to_18_detail_large.shape)
print("df_19_to_48_detail_large.shape",df_19_to_48_detail_large.shape)

df_0_to_18_detail_test_only.shape (101201, 3)
df_19_to_48_detail_test_only.shape (0, 3)
df_0_to_18_detail_large.shape (17753200, 3)
df_19_to_48_detail_large.shape (4915668, 3)


In [30]:
print("print(df_0_to_18_detail_test_only['segment'].nunique())",df_0_to_18_detail_test_only['segment'].nunique())

count_control_0_to_18=500000
count_control_19_to_48=100000
pctg_0_18=np.round(count_control_0_to_18/df_0_to_18_detail_large.shape[0],6)
print("pctg_0_18",pctg_0_18)
pctg_19_48=np.round(count_control_19_to_48/df_19_to_48_detail_large.shape[0],6)
print("pctg_19_48",pctg_19_48)

print(df_0_to_18_detail_test_only['segment'].nunique()) 112
pctg_0_18 0.028164
pctg_19_48 0.020343


In [31]:
writer_folder_all_wanted="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/wanted_segment_files/"
try:
    os.stat(writer_folder_all_wanted)
except:
    os.mkdir(writer_folder_all_wanted)


In [32]:
print(df_0_to_18_detail_large['segment'].nunique())
print(df_19_to_48_detail_large['segment'].nunique())
print(df_0_to_18_detail_test_only['segment'].nunique())


271
8
112


In [34]:
'''
df_count_aggreted=df_0_to_18_detail_test_only.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_count_aggreted=df_count_aggreted.sort_values("customer_id_hashed")
df_count_aggreted
'''
df_0_to_18_detail_test_only['segment']="OthersAggregated_RFM20Q1"
df_0_to_18_detail_large=df_0_to_18_detail_large.append(df_0_to_18_detail_test_only)
del df_0_to_18_detail_test_only
# Confirmed with YG to aggreate all together as 1 seg

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
print(datetime.datetime.now(),"start splitting")
random.seed(0)

df_test_all=pd.DataFrame()
df_control_all=pd.DataFrame()

for seg, group in df_0_to_18_detail_large.groupby("segment"):
    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    len_group=len(group)
    
    ind_control=random.sample(range(len_group),int(len_group*pctg_0_18))
    df_group_T=group[~group['index'].isin(ind_control)]
    df_group_C=group[group['index'].isin(ind_control)]
    
    del df_group_T['index']
    del df_group_C['index']
    
    new_T_seg="T_"+seg
    new_C_seg="C_"+seg
    
    df_group_T['segment']=new_T_seg
    df_group_C['segment']=new_C_seg
    
    df_test_all=df_test_all.append(df_group_T)
    df_control_all=df_control_all.append(df_group_C)
    
    df_group_T.to_csv(writer_folder_all_wanted+new_T_seg+".csv",index=False)
    df_group_C.to_csv(writer_folder_all_wanted+new_C_seg+".csv",index=False)
    print(new_T_seg,df_group_T.shape)
    print(new_C_seg,df_group_C.shape)
    del df_group_T
    del df_group_C
    
for seg, group in df_19_to_48_detail_large.groupby("segment"):
    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    len_group=len(group)
    
    ind_control=random.sample(range(len_group),int(len_group*pctg_19_48))
    df_group_T=group[~group['index'].isin(ind_control)]
    df_group_C=group[group['index'].isin(ind_control)]
    
    del df_group_T['index']
    del df_group_C['index']
    
    new_T_seg="T_"+seg
    new_C_seg="C_"+seg
    
    df_group_T['segment']=new_T_seg
    df_group_C['segment']=new_C_seg
    
    df_test_all=df_test_all.append(df_group_T)
    df_control_all=df_control_all.append(df_group_C)
    
    df_group_T.to_csv(writer_folder_all_wanted+new_T_seg+".csv",index=False)
    df_group_C.to_csv(writer_folder_all_wanted+new_C_seg+".csv",index=False)

    print(new_T_seg,df_group_T.shape)
    print(new_C_seg,df_group_C.shape)
    
    del df_group_T
    del df_group_C
'''
for seg,group in df_0_to_18_detail_test_only.groupby("segment"):
    new_TOnly_seg="TOnly_"+seg
    group['segment']=new_TOnly_seg
    
    df_test_all=df_test_all.append(group)
    
    group.to_csv(writer_folder_all_wanted+new_TOnly_seg+".csv",index=False)
    print(new_TOnly_seg,group.shape)
'''
# df_19_to_48_detail_test_only.shape (0, 3)
# So no need to iter overl 19_48 test only

# The smalls aggregated together

print("df_test_all.shape",df_test_all.shape)

print("df_control_all.shape",df_control_all.shape)

print(datetime.datetime.now(),"done splitting")

2020-01-28 16:06:09.694276 start splitting


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (30937, 3)
C_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (896, 3)
T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (48017, 3)
C_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (1391, 3)
T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (24033, 3)
C_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (696, 3)
T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (22194, 3)
C_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (643, 3)
T_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (628754, 3)
C_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (18221, 3)
T_Legacy_Active_H_P_Above50K_SingleFamily_Comp

T_Legacy_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (152503, 3)
C_Legacy_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (4419, 3)
T_Legacy_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (206261, 3)
C_Legacy_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (5977, 3)
T_Legacy_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (87138, 3)
C_Legacy_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (2525, 3)
T_Legacy_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (20430, 3)
C_Legacy_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (592, 3)
T_Legacy_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (13881, 3)
C_Legacy_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (402, 3)
T_Legacy_Active_L_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_Furinitu

T_Legacy_Active_M_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (28442, 3)
C_Legacy_Active_M_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (824, 3)
T_Legacy_Active_M_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (15072, 3)
C_Legacy_Active_M_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1 (436, 3)
T_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (144396, 3)
C_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (4184, 3)
T_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (53774, 3)
C_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (1558, 3)
T_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (127883, 3)
C_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (3706, 3)
T_Legacy_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_

T_Legacy_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (27244, 3)
C_Legacy_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (789, 3)
T_Legacy_Lapsed1318_L_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1 (4283, 3)
C_Legacy_Lapsed1318_L_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1 (124, 3)
T_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (6239, 3)
C_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (180, 3)
T_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (6695, 3)
C_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1 (193, 3)
T_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (4746, 3)
C_Legacy_Lapsed1318_L_S_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (137, 3)
T_Legacy_Lapsed1318_L_S_Above50K_Multi

T_SOTF_Active_H_S_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (121665, 3)
C_SOTF_Active_H_S_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (3525, 3)
T_SOTF_Active_H_S_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (23590, 3)
C_SOTF_Active_H_S_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (683, 3)
T_SOTF_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (7466, 3)
C_SOTF_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1 (216, 3)
T_SOTF_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (5861, 3)
C_SOTF_Active_H_S_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (169, 3)
T_SOTF_Active_H_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (34494, 3)
C_SOTF_Active_H_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (999, 3)
T_SOTF_Active_H_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (4048, 3)
C_SOTF

T_SOTF_Active_M_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (12252, 3)
C_SOTF_Active_M_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1 (355, 3)
T_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (87331, 3)
C_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1 (2530, 3)
T_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (10800, 3)
C_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1 (312, 3)
T_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (57046, 3)
C_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1 (1653, 3)
T_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (7127, 3)
C_SOTF_Active_M_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1 (206, 3)
T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1 (191755, 3)
C_SOTF_Active_M_P_Unkn

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


T_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureHigh_RFM20Q1 (1257156, 3)
C_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureHigh_RFM20Q1 (26105, 3)
T_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureLow_RFM20Q1 (656266, 3)
C_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureLow_RFM20Q1 (13627, 3)
T_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureHigh_RFM20Q1 (941581, 3)
C_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureHigh_RFM20Q1 (19552, 3)
T_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureLow_RFM20Q1 (339958, 3)
C_Legacy_Lapsed1948_NA_P_NA_NA_CompetitorsLow_FurinitureLow_RFM20Q1 (7059, 3)
T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureHigh_RFM20Q1 (935353, 3)
C_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureHigh_RFM20Q1 (19422, 3)
T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureLow_RFM20Q1 (180867, 3)
C_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsHigh_FurinitureLow_RFM20Q1 (3755, 3)
T_SOTF_Lapsed1948_NA_P_NA_NA_CompetitorsL

In [48]:
df_count_after_split_T=df_test_all.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_count_after_split_T=df_count_after_split_T.rename(columns={"customer_id_hashed":"test_ids"})
df_count_after_split_T['segment_T_name']=df_count_after_split_T['segment']
df_count_after_split_T['segment']=df_count_after_split_T['segment_T_name'].apply(lambda x: "_".join(x.split("_")[1:]))

df_count_after_split_C=df_control_all.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_count_after_split_C=df_count_after_split_C.rename(columns={"customer_id_hashed":"control_ids"})
df_count_after_split_C['segment_C_name']=df_count_after_split_C['segment']
df_count_after_split_C['segment']=df_count_after_split_C['segment_C_name'].apply(lambda x: "_".join(x.split("_")[1:]))

In [49]:
df_0_to_18_count_all=df_0_to_18_detail.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_19_to_48_count_all=df_19_to_48_detail.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_0_to_18_count_all=df_0_to_18_count_all.sort_values("segment")
df_19_to_48_count_all=df_19_to_48_count_all.sort_values("segment")

for seg_small in list_seg_small_0_to_18:
    df_0_to_18_count_all['segment']=df_0_to_18_count_all['segment'].replace(seg_small,"OthersAggregated_RFM20Q1")
    df_19_to_48_count_all['segment']=df_19_to_48_count_all['segment'].replace(seg_small,"OthersAggregated_RFM20Q1")
df_0_to_18_count_all=df_0_to_18_count_all.groupby("segment")['customer_id_hashed'].sum().to_frame().reset_index()
df_19_to_48_count_all=df_19_to_48_count_all.groupby("segment")['customer_id_hashed'].sum().to_frame().reset_index()
    
df_0_to_18_count_all=pd.merge(df_0_to_18_count_all,df_count_after_split_T,on="segment",how="left")
df_0_to_18_count_all=pd.merge(df_0_to_18_count_all,df_count_after_split_C,on="segment",how="left")
df_0_to_18_count_all['control_ids']=df_0_to_18_count_all['control_ids'].fillna(0)

df_19_to_48_count_all=pd.merge(df_19_to_48_count_all,df_count_after_split_T,on="segment",how="left")
df_19_to_48_count_all=pd.merge(df_19_to_48_count_all,df_count_after_split_C,on="segment",how="left")
df_19_to_48_count_all['control_ids']=df_19_to_48_count_all['control_ids'].fillna(0)


In [50]:
writer=pd.ExcelWriter("./BL_audience_count_summary_2020Q1_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_0_to_18_count_all.to_excel(writer,"df_0_to_18_count_all",index=False)
df_19_to_48_count_all.to_excel(writer,"df_19_to_48_count_all",index=False)
writer.save()

In [51]:
list_all_audience_local_files=glob.glob(writer_folder_all_wanted+"*.csv")

print("len(list_all_audience_local_files)",len(list_all_audience_local_files))

len(list_all_audience_local_files) 560


In [56]:
print(df_control_all.shape)
df_control_all.to_csv(writer_folder_all_wanted+"C_all_quarterly_segments_RFM20Q1"+".csv",index=False)

(602714, 3)


In [57]:
df_test_all.shape

# (22170205, 3)

(22167355, 3)

# Uploading

In [58]:
# Uploading in another script

last_quarter_df_uploaded=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/upload_files/C_Active(0-12)_Score(H)_Density(H)_Legacy_HolidayShoppers_S_2019Q4_RFM.csv",dtype=str,nrows=10)
last_quarter_df_uploaded.columns.tolist()


['customer_id_hashed', 'email_address_hash', 'segment']